In [8]:
from pymongo import MongoClient
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [9]:
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("companies")

In [10]:
condition_1 = {"total_money_raised": {"$regex": ".*[MB].*"}}
condition_2 = {"tag_list": {"$regex": ".*desi.*"}}

projection = {"name": 1, "_id": 0, "total_money_raised": 1, "offices" : 1}

results_1 = list(c.find(condition_1, projection))
results_2 = list(c.find(condition_2, projection))

In [11]:
df_1 = pd.DataFrame(results_1)
df_2 = pd.DataFrame(results_2)

df_1['condition'] = 'condition_1'
df_2['condition'] = 'condition_2'

df = pd.concat([df_1, df_2], axis=0)

df_unique = df.drop_duplicates(subset=['condition', 'name'])

In [12]:
def extraer_columnas(row):
    if row['offices']:
        
        office = row['offices'][0]
       
        row['city'] = office.get('city', None)
        row['country'] = office.get('country_code', None)
        row['cp'] = office.get('zip_code', None)
        row['latitude'] = office.get('latitude')
        row['longitude'] = office.get('longitude')
    else:
        row['city'] = None
        row['country'] = None
        row['cp'] = None
        row['latitude'] = None
        row['longitude'] = None
    return row


df_2 = df.apply(extraer_columnas, axis=1)


df_2.drop('offices', axis=1, inplace=True)
df_2.drop('total_money_raised', axis=1, inplace=True)
df_2.dropna(subset=['latitude', 'longitude'], inplace=True)


In [13]:

mapa = folium.Map(location=[df_2['latitude'].iloc[0], df_2['longitude'].iloc[0]], zoom_start=2)


heat_data = [[row['latitude'], row['longitude']] for index, row in df_2.iterrows() if row['latitude'] is not None and row['longitude'] is not None]
HeatMap(heat_data).add_to(mapa)


mapa

In [14]:
mapa = folium.Map(location=[df_2['latitude'].iloc[0], df_2['longitude'].iloc[0]], zoom_start=2)

for index, row in df_2.iterrows():
    if row['latitude'] is not None and row['longitude'] is not None:
        folium.Marker([row['latitude'], row['longitude']]).add_to(mapa)
mapa